# Convoltioin Neural Network

In [1]:
# importing necessary libraries
import numpy as np

import torch as tr
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

In [2]:
# device configuration
device = tr.device('cuda' if tr.cuda.is_available() else 'cpu')

In [3]:
# hyper parameters
# input_size = 784 #28X28
# hidden_size = 100
num_classes = 10

num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [4]:
# datset has PILImage images of range [0,1]
# we transform those images to Tensors of normalized range [-1,1]
transfrom = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

In [5]:
# MNIST
train_dataset = torchvision.datasets.CIFAR10(root = './data_CIFAR',
    train = True, transform= transfrom, download= True)

test_dataset = torchvision.datasets.CIFAR10(root = './data_CIFAR',
    train= False, transform= transfrom, download= True)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data_CIFAR/cifar-10-python.tar.gz to ./data_CIFAR
Files already downloaded and verified


In [6]:
train_loader = tr.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = tr.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [7]:
classses = ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [10]:
# CNN layer output shape calculation
def imshow(img):
    img = img/2 + 0.5
    npimg = img.numpy()
    # plt.imshow(np.transpose(npimg, (1,2,0)))
    # plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))

conv1 = nn.Conv2d(3,6,5)
pool = nn.MaxPool2d(2,2)
conv2 = nn.Conv2d(6,16,5)
print(images.shape)

x = conv1(images)
print(x.shape)
x = pool(x)
print(x.shape)
x = conv2(x)
print(x.shape)
x = pool(x)
print(x.shape)


torch.Size([4, 3, 32, 32])
torch.Size([4, 6, 28, 28])
torch.Size([4, 6, 14, 14])
torch.Size([4, 16, 10, 10])
torch.Size([4, 16, 5, 5])


In [11]:
# Implementation of ConvNet using pytorch
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



conv_model = ConvNet().to(device)

criteria = nn.CrossEntropyLoss()
optimizer = tr.optim.SGD(conv_model.parameters(), lr= learning_rate)



In [13]:
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 100,1,28,28 --> 100, 784
        images = images.to(device)
        labels = labels.to(device)

        # forward
        outputs = conv_model(images)
        loss = criteria(outputs, labels)

        # backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0 :
            print(f'Epoch {epoch+1} / {num_epochs}, Step [{i+1}/{n_total_steps}], Loss  = {loss.item(): .4f}')

print('Finished training')

# test 
with tr.no_grad():
    no_correct = 0
    n_samples = 0
    n_classes_correct = [0 for i in range(10)]
    n_classes_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = conv_model(images)

        #value, index
        _,predictions = tr.max(outputs, 1)
        n_samples += labels.size(0)
        no_correct += (predictions == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predictions[i]
            if label == pred :
                n_classes_correct[label] += 1
            n_classes_samples[label] += 1

    
    accuracy = 100.0 * no_correct/n_samples
    print(f'Accuracy of the network = {accuracy}% ')

    for i in range(10):
        acc = 100.0 * n_classes_correct[i]/ n_classes_samples[i]
        print(f'Accuracy of {classses[i]}: {acc}% ')

Epoch 1 / 4, Step [2000/12500], Loss  =  2.3204
Epoch 1 / 4, Step [4000/12500], Loss  =  2.3160
Epoch 1 / 4, Step [6000/12500], Loss  =  2.2946
Epoch 1 / 4, Step [8000/12500], Loss  =  2.2763
Epoch 1 / 4, Step [10000/12500], Loss  =  2.1781
Epoch 1 / 4, Step [12000/12500], Loss  =  2.1148
Epoch 2 / 4, Step [2000/12500], Loss  =  2.1171
Epoch 2 / 4, Step [4000/12500], Loss  =  1.8656
Epoch 2 / 4, Step [6000/12500], Loss  =  2.4000
Epoch 2 / 4, Step [8000/12500], Loss  =  1.2929
Epoch 2 / 4, Step [10000/12500], Loss  =  1.4138
Epoch 2 / 4, Step [12000/12500], Loss  =  1.7204
Epoch 3 / 4, Step [2000/12500], Loss  =  1.5130
Epoch 3 / 4, Step [4000/12500], Loss  =  1.2996
Epoch 3 / 4, Step [6000/12500], Loss  =  2.1206
Epoch 3 / 4, Step [8000/12500], Loss  =  1.8350
Epoch 3 / 4, Step [10000/12500], Loss  =  0.9485
Epoch 3 / 4, Step [12000/12500], Loss  =  1.3501
Epoch 4 / 4, Step [2000/12500], Loss  =  1.3403
Epoch 4 / 4, Step [4000/12500], Loss  =  1.2171
Epoch 4 / 4, Step [6000/12500], Lo